In [3]:
pip install pandas numpy matplotlib keras

  Using cached matplotlib-3.10.6-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached keras-3.11.3-py3-none-any.whl.metadata (5.9 kB)
Using cached matplotlib-3.10.6-cp311-cp311-win_amd64.whl (8.1 MB)
Using cached keras-3.11.3-py3-none-any.whl (1.4 MB)

   ---------------------------------------- 0/2 [matplotlib]
   ---------------------------------------- 0/2 [matplotlib]
   ---------------------------------------- 0/2 [matplotlib]
   ---------------------------------------- 0/2 [matplotlib]
   ---------------------------------------- 0/2 [matplotlib]
   ---------------------------------------- 0/2 [matplotlib]
   ---------------------------------------- 0/2 [matplotlib]
   ---------------------------------------- 0/2 [matplotlib]
   ---------------------------------------- 0/2 [matplotlib]
   ---------------------------------------- 0/2 [matplotlib]
   ---------------------------------------- 0/2 [matplotlib]
   ---------------------------------------- 0/2 [matplotlib]
   -------

In [4]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import keras
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

loading our image to data generator

loading a pre trained model

In [ ]:
train_datagen = ImageDataGenerator(zoom_range= 0.5, shear_range= 0.3, horizontal_flip = True, preprocessing_function= preprocess_input)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

Preprocessing

In [ ]:
train = train_datagen.flow_from_directory('Path of the train_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32
                                          )

val = val_datagen.flow_from_directory('Path of the test_set',
                                                 target_size = (224, 224),
                                                 batch_size = 16
                                      )


In [ ]:
t_img, label = train.next()

In [ ]:
def plotImage(img_arr, label):
  for im , l in zip(img_arr, label):
    plt.figure(figsize=(5,5))
    plt.imshow(im/200)
    plt.show()

In [ ]:
plotImage(t_img[:3], label[:3])

Building a model

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions
import keras

In [ ]:
base_model = VGG19(input_shape=(224,224,3), include_top= False)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False


In [ ]:
base_model.summary()

In [ ]:
x = Flatten()(base_model.output)
x = Dense(units= 8 , activation = 'softmax')(x)
model = Model(base_model.input, x)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy , metrics= ['accuracy'])

Early stopping and checkpoint

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

es = EarlyStopping(monitor='accuracy', min_delta= 0.01, patience = 3, verbose = 1)

mc = ModelCheckpoint(filepath="Edureka.h5",
                     monitor='accuracy',
                     min_delta= 0.01,
                     patience = 3,
                     verbose = 1,
                     save_best_only= True)
cb = [es, mc]

training the model

In [ ]:
his = model.fit_generator(train ,
                         steps_per_epoch = 16,
                         epochs= 75,
                         verbose=1,
                         callbacks= cb,
                         validation_data= val,
                         validation_steps =16)

In [ ]:
h = his.history
h.keys()


In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c = "red")
plt.title("accuracy vs v-accuracy")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c = "red")
plt.title("loss vs v-loss")
plt.show()

In [ ]:
model.save("models/skin_model.h5")

///

In [ ]:
from keras.models import load_model
model = load_model("Path of the model")

In [ ]:
acc = model.evaluate_generator(val)[1]
print(f"the acc is {acc*100}%")

In [ ]:
ref = dict(zip(list(train.class_indices.values()), list(train.class_indices.keys())))

In [ ]:
def prediction(path):
  img = load_img(path, target_size= (224,224))
  i = img_to_array(img)
  im = preprocess_input(i)
  img = np.expand_dims(im, axis = 0)
  pred = np.argmax(model.predict(img))
  print(f"the image is {ref[pred]}")

upload the image to google and copy the path and paste in the path.

In [ ]:
img_path= "Path of the image to detected"
prediction(img_path)





In [ ]:
import tensorflow as tf

# Define a new Model, Input= image
# Output= intermediate representations for all layers in the
# previous model after the first.

successive_outputs = [layer.output for layer in model.layers[1:]]

#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

#Load the input image
img = load_img(img_path, target_size=(224, 224))

# Convert ht image to Array of dimension (150,150,3)
x   = img_to_array(img)
x   = x.reshape((1,) + x.shape)

# Rescale by 1/255
x /= 224.0

# Let's run input image through our vislauization network
# to obtain all intermediate representations for the image.
successive_feature_maps = visualization_model.predict(x)

# Retrieve are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  print(feature_map.shape)
  if len(feature_map.shape) == 4:

    # Plot Feature maps for the conv / maxpool layers, not the fully-connected layers

    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 2]  # feature map shape (1, size, size, n_features)

    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))

    # Postprocess the feature to be visually palatable
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')

      # Tile each filter into a horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x

# Display the grid
    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' )